# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

The principal owns a Joe and the Juice and employ an employee (Agent) to sell the juice. The agents utility function is given by $\frac{w^{1-\rho}}{1-\rho} - c \cdot e$, where w is the wage received, and $e=e^*$ when the employee stays home at night to be well-rested and $e=0$ when he goes partying. He has a reservation utility of $u$.

The employee can sell lots of juice and only a little juice. When the employee is hungover the probability of high juice sales is $\pi_0$. When the employee is not hungover, $e=e^*$ the probability is $\pi_1$.

The principal suggest a contract with a wage og $w_H$ if the employee sells a lot, and $w_L$ if the employee sells a little. 
The principal chooses $(w_L,w_H)$ so as to minimize expected wage payment and wants the employee to work hard:
$$\begin{align} \pi_e * w_H +(1-\pi_1)w_L
\end{align}$$

So the problem looks like the following:
$$ 
\begin{align}
\min_{w_{h},w_{L}} \pi_e * w_H +(1-\pi_1)w_L \\
 \text{s.t.} \\
 \pi_1 (\frac{w_H^{1-\rho}}{1-\rho} -c \cdot e^*) + (1-\pi_1) (\frac{w_L^{1-\rho}}{1-\rho} -c \cdot e^*) \geq \bar{u}
\end{align}

$$

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 